**Задание1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

***Решение***

In [1]:
import pandas as pd

In [101]:
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep=',')

ratings_count = ratings.groupby('userId').count().reset_index()
ratings_count = ratings_count[['userId','rating']]

more_than_100 = ratings_count[ratings_count['rating'] > 100]
more_than_100.rename(columns = {'rating': 'rating_count'}, inplace=True)

rating_max = ratings.groupby('userId').max().reset_index()
rating_min = ratings.groupby('userId').min().reset_index()

max_timestamp = rating_max[['userId','timestamp']]
min_timestamp = rating_min[['userId','timestamp']]

max_timestamp.rename(columns = {'timestamp': 'max_time'}, inplace=True)
min_timestamp.rename(columns = {'timestamp': 'min_time'}, inplace=True)

avg_timestamp = more_than_100.merge(max_timestamp).merge(min_timestamp)

def average(row):
    result = (row['max_time'] - row['min_time'])
    return result

avg_timestamp['average'] = avg_timestamp.apply(average, axis=1)

avg_timestamp.head()

,userId,rating_count,max_time,min_time,average
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282


**Задание2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

***Решение***

In [102]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [103]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [104]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [105]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [217]:
revenue_table = rzd.merge(auto, how='outer', on='client_id')

revenue_table = revenue_table.merge(air, how='outer', on='client_id')

revenue_table.loc[revenue_table.rzd_revenue.isnull(), 'rzd_revenue',] = 0
revenue_table.loc[revenue_table.auto_revenue.isnull(), 'auto_revenue',] = 0
revenue_table.loc[revenue_table.air_revenue.isnull(), 'air_revenue',] = 0

full_table = revenue_table.merge(client_base, how='outer', on='client_id')

revenue_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [218]:
full_table

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


**Задание3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

***Решение***

1. Зная координаты места покупки, можно построить вектора пользователя и магазина и ешли будет минимальный угол - знаит это максимальное совпадение и пользователь посетил магазин.
2. Расчитывая разные координаты, мы поймем в какие из дней пользователь посещал магазин, что также поможет делать те или иные акции в момент, когда вероятность пользователя посетить магазин и купить что-то велика.
3. Можно понять где больше всего времени проводит пользователь, как быстро он передвигается (и предложить ему что-то для пешей прогулки, езды на велосипеде или товаров для авто).

In [228]:
client_base = pd.DataFrame(
    {
        'user_id': [123, 123, 345],
        'user_coordinates': ['55.658996099428364, 37.73254394531251', '55.67913303850629, 37.51831054687501', '55.67293816096562, 38.41094970703126'],
        'nearest_shop': ['Market', 'Shop', 'Supermarket']
    }
)
client_base

,user_id,user_coordinates,nearest_shop
0,123,"55.658996099428364, 37.73254394531251",Market
1,123,"55.67913303850629, 37.51831054687501",Shop
2,345,"55.67293816096562, 38.41094970703126",Supermarket
